# Evaluación final: Caso práctico
Cada día es más frecuente la introducción de la ciencia de datos en el ámbito del derecho y la justicia.

Un ejemplo bien conocido de ello es el sistema **COMPAS** (Correctional offender management profiling for alternative sanctions) que se usa en varios estados de los Estados Unidos para hacer una evaluación del riesgo de reincidencia de las personas detenidas.

Una breve descripción del sistema puede verse en la página https://en.wikipedia.org/wiki/COMPAS_(software).


En este caso, proporciona un conjunto de datos en bruto con información de las evaluaciones (fichero compas-scores.csv) y la historia legal de unos 11 000 casos en los años 2013 y 2014 (se trata de uno de los ficheros originales utilizados en un análisis independiente del sistema COMPAS llevado a cabo por ProPublica, disponible en internet).

Aunque el conjunto de datos contiene información adicional, para resolver las cuestiones planteadas en este caso son necesarios (aparte de algunos campos cuyo nombre es autoexplicativo) los siguientes campos:

1. **“compas_screening_date”**: se refiere a la fecha en la que se realizó la evaluación
2. **“decile_score”**: es un número, de 1 a 10 que indica el riesgo de reincidencia en general (a mayor riesgo, mayor número).
3. **“v_decile_score”**: es un número de 1 a 10, potencialmente distinto del anterior, que indica el riesgo de reincidencia en delitos violentos. Al hacer la evaluación de un caso en COMPAS, se generan las dos puntuaciones (entre otras cosas).
4. **“is_recid”**: indicación de si la persona es reincidente (en el tiempo en que se recogen datos: no hay información de si la persona es reincidente más allá de ciertas fechas, y es importante tener esto en cuenta para asegurarse de hacer comparaciones homogéneas).
5. **“r_offense_date”**: fecha en la que se cometió el delito por el que se considera reincidente a la persona.
6. **“is_violent_recid”**: indicación de si la persona es reincidente en un delito con violencia (las mismas consideraciones sobre fechas que para “is_recid” aplican aquí)".
7. **“vr_offense_date”**: fecha en la que se cometió el delito violento que da lugar a la consideración de reincidente.


## Se pide:
1. Cargar los datos y realizar un análisis exploratorio y una evaluación de la calidad de los datos necesarios para el resto del caso. Específicamente, evaluar la integridad, validez y actualidad de los datos y proponer estrategias de mitigación de los posibles problemas encontrados.

2. ¿Son los campos “is_recid” e “is_violent_recid” en este conjunto de datos adecuados para evaluar la precisión de las estimaciones de riesgo generadas por el sistema COMPAS? Si no es así, definir y calcular una feature que sí lo sea.

3. El umbral para establecer medidas preventivas de la reincidencia es de 7 en adelante.Dado este umbral, generar una tabla de contingencia, explicando qué caso se considera como “positivo” (y, por lo tanto, cuáles son los errores de tipo I y los errores de tipo II).

4. El sistema asigna, de media, evaluaciones de riesgo más altas a los hombres que a las mujeres, y a las personas de raza afroamericana que a las de raza caucásica. Sin embargo, también las tasas de reincidencia son más altas para esos colectivos, aunque no está claro que la asignación de riesgo sea “justa” o no. Mostrar estas diferencias mediante representaciones gráficas y utilizarlas para analizar si la asignación de evaluaciones es justa o no. 

5. ¿Para qué tipo de riesgos, el de delitos generales o el de delitos violentos, tiene el sistema más capacidad predictiva?

## Se pide
### 1. Cargar los datos y realizar un análisis exploratorio y una evaluación de la calidad de los datos necesarios para el resto del caso. Específicamente, evaluar la integridad, validez y actualidad de los datos y proponer estrategias de mitigación de los posibles problemas encontrados.



In [1]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn import metrics
import matplotlib.pyplot as plt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
#Se cargan los datos del CSV a un DataFrame de Pandas.
compas = pd.read_csv('./datos/compas-scores.csv')

In [3]:
compas.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [4]:
compas.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11757 entries, 0 to 11756
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       11757 non-null  int64  
 1   name                     11757 non-null  object 
 2   first                    11757 non-null  object 
 3   last                     11757 non-null  object 
 4   compas_screening_date    11757 non-null  object 
 5   sex                      11757 non-null  object 
 6   dob                      11757 non-null  object 
 7   age                      11757 non-null  int64  
 8   age_cat                  11757 non-null  object 
 9   race                     11757 non-null  object 
 10  juv_fel_count            11757 non-null  int64  
 11  decile_score             11757 non-null  int64  
 12  juv_misd_count           11757 non-null  int64  
 13  juv_other_count          11757 non-null  int64  
 14  priors_count          

In [5]:
#Se eliminarán las columnas vacías.
compas = compas.drop(['num_vr_cases', 'num_r_cases'], axis = 1)

In [6]:
#Se creará un dataframe reducido con los campos que se indican en el enunciado además de los campos que caracterizarán al individuo ('name', 'age', 'sex' y 'race') y 'priors_count', el número de detenciones previas.
compas_red = compas[['name', 'age', 'dob','sex', 'race', 'compas_screening_date', 'decile_score', 'v_decile_score', 'is_recid', 'r_offense_date', 'is_violent_recid', 'vr_offense_date', 'priors_count']].copy()

In [7]:
compas_red.head()

,name,age,dob,sex,race,compas_screening_date,decile_score,v_decile_score,is_recid,r_offense_date,is_violent_recid,vr_offense_date,priors_count
0,miguel hernandez,69,1947-04-18,Male,Other,2013-08-14,1,1,0,NaN,0,NaN,0
1,michael ryan,31,1985-02-06,Male,Caucasian,2014-12-31,5,2,-1,NaN,0,NaN,0
2,kevon dixon,34,1982-01-22,Male,African-American,2013-01-27,3,1,1,2013-07-05,1,2013-07-05,0
3,ed philo,24,1991-05-14,Male,African-American,2013-04-14,4,3,1,2013-06-16,0,NaN,4
4,marcu brown,23,1993-01-21,Male,African-American,2013-01-13,8,6,0,NaN,0,NaN,1


In [8]:
#Se trabajará con este dataframe reducido.
compas_red.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11757 entries, 0 to 11756
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   11757 non-null  object
 1   age                    11757 non-null  int64 
 2   dob                    11757 non-null  object
 3   sex                    11757 non-null  object
 4   race                   11757 non-null  object
 5   compas_screening_date  11757 non-null  object
 6   decile_score           11757 non-null  int64 
 7   v_decile_score         11757 non-null  int64 
 8   is_recid               11757 non-null  int64 
 9   r_offense_date         3703 non-null   object
 10  is_violent_recid       11757 non-null  int64 
 11  vr_offense_date        882 non-null    object
 12  priors_count           11757 non-null  int64 
dtypes: int64(6), object(7)
memory usage: 1.2+ MB


In [9]:
 compas_red.nunique()

name                     11584
age                         66
dob                       7800
sex                          2
race                         6
compas_screening_date      704
decile_score                11
v_decile_score              11
is_recid                     3
r_offense_date            1090
is_violent_recid             2
vr_offense_date            599
priors_count                39
dtype: int64

In [10]:
compas_red.describe(include = 'object')

,name,dob,sex,race,compas_screening_date,r_offense_date,vr_offense_date
count,11757,11757,11757,11757,11757,3703,882
unique,11584,7800,2,6,704,1090,599
top,carlos vasquez,1992-10-15,Male,African-American,2013-03-20,2014-12-08,2015-08-15
freq,4,6,9336,5813,39,12,6


In [11]:
#Conversión de tipos.
#Se pasarán los nombres a string.
compas_red['name'] = compas_red['name'].astype('string')
#Se pasan a datetime las columnas que contienen fechas.
compas_red.dob = pd.to_datetime(compas_red.dob, format="%Y-%m-%d")
compas_red.compas_screening_date = pd.to_datetime(compas_red.compas_screening_date, format="%Y-%m-%d")
compas_red.r_offense_date = pd.to_datetime(compas_red.r_offense_date, format="%Y-%m-%d")
compas_red.vr_offense_date = pd.to_datetime(compas_red.vr_offense_date, format="%Y-%m-%d")
#El resto se pasan a category.
compas_red['sex'] = compas_red['sex'].astype('category')
compas_red['race'] = compas_red['race'].astype('category')
compas_red['is_recid'] = compas_red['is_recid'].astype('category')
compas_red['is_violent_recid'] = compas_red['is_violent_recid'].astype('category')

In [12]:
#se comprueba que no se han introducido más nulos.
compas_red.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11757 entries, 0 to 11756
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   name                   11757 non-null  string        
 1   age                    11757 non-null  int64         
 2   dob                    11757 non-null  datetime64[ns]
 3   sex                    11757 non-null  category      
 4   race                   11757 non-null  category      
 5   compas_screening_date  11757 non-null  datetime64[ns]
 6   decile_score           11757 non-null  int64         
 7   v_decile_score         11757 non-null  int64         
 8   is_recid               11757 non-null  category      
 9   r_offense_date         3703 non-null   datetime64[ns]
 10  is_violent_recid       11757 non-null  category      
 11  vr_offense_date        882 non-null    datetime64[ns]
 12  priors_count           11757 non-null  int64         
dtypes

In [13]:
compas_red.describe(include='category')

,sex,race,is_recid,is_violent_recid
count,11757,11757,11757,11757
unique,2,6,3,2
top,Male,African-American,0,0
freq,9336,5813,7335,10875


Una vez realizados los cambios en el dataframe, se procederá a analizar las columnas para evaluar la integridad, validez y actualidad. 

In [14]:
def describe_columna(df, col):
    print(f'Columna: {col}  -  Tipo de datos: {df[col].dtype}')
    print(f'Número de valores nulos: {df[col].isnull().sum()}  -  Número de valores distintos: {df[col].nunique()}')
    print('Valores más frecuentes:')
    for i, v in df[col].value_counts().iloc[:10].items() :
        print(i, '\t', v)

In [15]:
describe_columna(compas_red, 'name')

Columna: name  -  Tipo de datos: string
Número de valores nulos: 0  -  Número de valores distintos: 11584
Valores más frecuentes:
carlos vasquez 	 4
john brown 	 4
robert taylor 	 4
michael cunningham 	 4
steven wilson 	 3
michael williams 	 3
richard jones 	 3
james smith 	 3
anthony jackson 	 3
james williams 	 3


In [16]:
#Se puede ver que existen nombres duplicados (se incluirán duplicidades por fecha de nacimiento para que sean exactamente la misma persona y así eliminar coincidencias de nombre), habrá que investigar el motivo de ello para ver si es sólo coincidencia o algún error.
compas_red[compas_red.duplicated(['name', 'dob'],keep = False)].sort_values(by = 'name')

,name,age,dob,sex,race,compas_screening_date,decile_score,v_decile_score,is_recid,r_offense_date,is_violent_recid,vr_offense_date,priors_count
6536,alejandro cabrera,26,1990-03-30,Male,Caucasian,2013-09-23,4,3,1,2013-10-06,1,2013-10-06,4
11585,alejandro cabrera,26,1990-03-30,Male,Hispanic,2013-10-07,4,3,-1,NaT,0,NaT,0
604,gueslly deravine,26,1989-11-30,Male,African-American,2013-03-27,7,7,0,NaT,0,NaT,6
3761,gueslly deravine,26,1989-11-30,Male,African-American,2013-02-21,5,6,-1,NaT,0,NaT,0
3701,scott botkin,46,1969-10-05,Male,Caucasian,2013-02-08,8,3,1,2014-01-04,0,NaT,5
9956,scott botkin,46,1969-10-05,Male,Caucasian,2013-06-18,3,2,-1,NaT,0,NaT,0


Como se puede apreciar no se trata de errores en el dataframe sino que se les han realizado varias evaluaciones en diferentes fechas. Por tanto no son duplicados y no se eliminaran. 

In [17]:
#Columna edad
describe_columna(compas_red, 'age')
compas_red.age.describe()

Columna: age  -  Tipo de datos: int64
Número de valores nulos: 0  -  Número de valores distintos: 66
Valores más frecuentes:
26 	 540
24 	 539
25 	 521
23 	 507
27 	 506
22 	 503
21 	 500
29 	 456
30 	 441
28 	 430


count    11757.000000
mean        35.143319
std         12.022894
min         18.000000
25%         25.000000
50%         32.000000
75%         43.000000
max         96.000000
Name: age, dtype: float64

En la columna de edad no existen valores nulos y además, todos los valores se encuentran en los rangos de 18 a 96 años, por tanto la columna está correcta

In [18]:
#Columna sexo
describe_columna(compas_red, 'sex')

Columna: sex  -  Tipo de datos: category
Número de valores nulos: 0  -  Número de valores distintos: 2
Valores más frecuentes:
Male 	 9336
Female 	 2421


La columna de sexo también se encuentra correctamente

In [19]:
#Columna raza
describe_columna(compas_red, 'race')

Columna: race  -  Tipo de datos: category
Número de valores nulos: 0  -  Número de valores distintos: 6
Valores más frecuentes:
African-American 	 5813
Caucasian 	 4085
Hispanic 	 1100
Other 	 661
Asian 	 58
Native American 	 40


La columna raza está correcta

In [20]:
#Se compprobarán las columnas 'is_recid', 'decile_score' y 'v_decile_score'. 
describe_columna(compas_red, 'is_recid')
compas_red.describe()

Columna: is_recid  -  Tipo de datos: category
Número de valores nulos: 0  -  Número de valores distintos: 3
Valores más frecuentes:
0 	 7335
1 	 3703
-1 	 719


,age,decile_score,v_decile_score,priors_count
count,11757.000000,11757.000000,11757.000000,11757.000000
mean,35.143319,4.371268,3.571489,3.082164
std,12.022894,2.877598,2.500479,4.687410
min,18.000000,-1.000000,-1.000000,0.000000
25%,25.000000,2.000000,1.000000,0.000000
50%,32.000000,4.000000,3.000000,1.000000
75%,43.000000,7.000000,5.000000,4.000000
max,96.000000,10.000000,10.000000,43.000000


En las columnas 'is_recid', 'decile_score' y 'v_decile_score'. 
Usualmente el valor -1 se utiliza para señalar la no reincidencia.
Por tanto, al utilizarse este valor para indicar que son casos para los que no existereincidencia, se eliminarán.

In [21]:
compas_red = compas_red[compas_red['is_recid'] != -1]
compas_red = compas_red[compas_red['decile_score'] != -1]
compas_red = compas_red[compas_red['v_decile_score'] != -1]

In [22]:
#Como se puede apreciar, aún existen valores nulos en las columnas 'r_offense_date' y en 'vr_offense_date' y se debe averiguar por qué.
compas_red.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11027 entries, 0 to 11756
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   name                   11027 non-null  string        
 1   age                    11027 non-null  int64         
 2   dob                    11027 non-null  datetime64[ns]
 3   sex                    11027 non-null  category      
 4   race                   11027 non-null  category      
 5   compas_screening_date  11027 non-null  datetime64[ns]
 6   decile_score           11027 non-null  int64         
 7   v_decile_score         11027 non-null  int64         
 8   is_recid               11027 non-null  category      
 9   r_offense_date         3701 non-null   datetime64[ns]
 10  is_violent_recid       11027 non-null  category      
 11  vr_offense_date        882 non-null    datetime64[ns]
 12  priors_count           11027 non-null  int64         
dtypes

In [23]:
describe_columna(compas_red, 'is_recid')

Columna: is_recid  -  Tipo de datos: category
Número de valores nulos: 0  -  Número de valores distintos: 2
Valores más frecuentes:
0 	 7326
1 	 3701
-1 	 0


In [24]:
describe_columna(compas_red, 'is_violent_recid')

Columna: is_violent_recid  -  Tipo de datos: category
Número de valores nulos: 0  -  Número de valores distintos: 2
Valores más frecuentes:
0 	 10145
1 	 882


Como se puede apreciar en la celda anterior, los nulos de 'r_offense_date' coinciden con los valores de negativos (no reincidente) de la columna is_recid, por tanto, si no es reincidente no puede existir fecha de reincidencia. Dee esos 3701, se concluye que 882 son delitos reincidentes violentos ('vr_offense_date). Columnas correctas.

In [25]:
#Análisis de fechas.
#Se visualizarám las fechas máximas y mínimas.
rd = compas_red.r_offense_date.min(), compas_red.r_offense_date.max()
vrd = compas_red.r_offense_date.min(), compas_red.r_offense_date.max()
csd = compas_red.compas_screening_date.min(), compas_red.compas_screening_date.max()

print('r_offense_date \n', {rd})
print('vr_offense_date \n', {vrd})
print('compas_screening_date \n', {csd})

r_offense_date 
 {(Timestamp('2013-01-03 00:00:00'), Timestamp('2016-03-29 00:00:00'))}
vr_offense_date 
 {(Timestamp('2013-01-03 00:00:00'), Timestamp('2016-03-29 00:00:00'))}
compas_screening_date 
 {(Timestamp('2013-01-01 00:00:00'), Timestamp('2014-12-31 00:00:00'))}


### 2. ¿Son los campos “is_recid” e “is_violent_recid” en este conjunto de datos adecuados para evaluar la precisión de las estimaciones de riesgo generadas por el sistema COMPAS? Si no es así, definir y calcular una feature que sí lo sea.